In [1]:
import pandas as pd 
import numpy as np 

from sys import exit, argv
import time

In [3]:
def id3(df, t, f):
	
	root, ig = {}, {}
	attr = df.columns.drop(t) 
	for a in attr:
		ig[a] = find_information_gain(df, t, a) 
	highest_ig = max(ig, key=lambda key: ig[key]) 
	s = make_split(df, highest_ig)
	root = {highest_ig:{}} 
	for v in s.keys(): 
		df_branch = df.where(df[highest_ig] == v).dropna() 
		
		if find_entropy(df_branch[t]) == 0:
		
			root[highest_ig][v] = s[v][t].value_counts().idxmax()
		else: 
			if len(attr) - 1 == 0: 
				
				root[highest_ig][v] = s[v][t].value_counts().idxmax()
				return root
			else: 
			
				root[highest_ig][v] = id3(s[v].drop(highest_ig, axis=1), t, f)
	return root
	
def find_entropy(t):
	
	h = 0
	v, n = np.unique(t, return_counts = True)
	for x in range(len(v)):
		px = n[x]/np.sum(n)
		h += -px * np.log2(px)
	return h
	
def find_information_gain(df, t, s):
	
	total_h = find_entropy(df[t]) 
	split_h = 0 
	v, n = np.unique(df[s], return_counts = True) 
	for x in range(len(v)):
		pt = n[x]/np.sum(n)
		split = df.where(df[s] == v[x]).dropna()[t] 
		split_h += pt * find_entropy(split)
	return total_h - split_h


In [4]:
def make_split(df, t):
	
	new_df = {}
	for df_key in df.groupby(t).groups.keys():
		new_df[df_key] = df.groupby(t).get_group(df_key)
	return new_df

def find_accuracy(dt, t):
	
	correct, total = 0, 0
	for _, e in t.iterrows():
		total += 1 
		if e[len(e)-1] == predict_decision(dt, e):
			correct += 1 
	return round(((correct/total)*100), 1)



In [5]:
def predict_decision(dt, e):
	
	split = list(dt.keys())[0]
	try:
		branch = dt[split][e[split]]
	except KeyError:
		
		return None
	if not isinstance(branch, dict): 
		return branch
	return predict_decision(branch, e) 

def holdout(df, p):
	
	if 0.00 < p < 1.00:
		d = df.copy()
		train = d.sample(frac=p) 
		test = d.drop(train.index) 
		if len(test) == 0:
			print("Proportion of training examples is too high.\n")
			exit(1)
		return train, test
	else:
		print("\nThe proportion of training examples must be (0.00..1.00).\n")
		exit(1)


In [6]:
def count_leaves(dt, c=[0,0]):
	
	c[0] += 1
	leaves = dt.keys()
	for leaf in leaves:
		branches = dt[leaf].values()
		for branch in branches:
			if isinstance(branch, dict):
				count_leaves(branch, c)
			else:
				c[1] += 1
	return c



In [7]:
def print_tree(dt, indent=0):
	
	for key, value in dt.items():
		print("  " * indent + str(key))
		if isinstance(value, dict): 
			print_tree(value, indent+1)
		else: # otherwise value
			print("  " * (indent+1) + str(value))


In [10]:
def print_statistics(dt, t, tr, te, trs, tes):
	
	s, d = count_leaves(dt) 
	print(f"Using {trs} training examples and {tes} testing examples.")
	print(f"Tree contains {s} non-leaf nodes and {d} leaf nodes.")
	print("Took {:.2f} seconds to generate.".format(t))
	print(f"Was able to classify {tr}% of training data.")
	print(f"Was able to classify {te}% of testing data.\n")


In [11]:
def load_csv(f):
	
	try:
		df = pd.read_csv('id3-train.csv', dtype=str) 
	except:
		print("\nData could not be loaded, ensure the arguments are correct.\n")
		exit(1)
	print(" was successfully loaded.")
	return df


In [12]:
def get_data():
  train = pd.read_csv('id3-train.csv')
  test  = pd.read_csv('id3-test.csv')
  return train, test

In [13]:
if __name__ == '__main__':
	print()
	train, test = get_data()
	decision_name = train.columns[len(train.columns)-1]
	start_time = time.time()
	dt = id3(train, decision_name, train.columns[:-1]) 
	end_time = time.time();
	
	t = end_time-start_time
	tr_size = len(train)
	te_size = len(test)
	tr_ability = find_accuracy(dt, train)
	te_ability = find_accuracy(dt, test)
	print_statistics(dt, t, tr_ability, te_ability, tr_size, te_size)
print("\n",  dt, "\n")
print_tree(dt)


Using 800 training examples and 203 testing examples.
Tree contains 58 non-leaf nodes and 34 leaf nodes.
Took 1.23 seconds to generate.
Was able to classify 55.2% of training data.
Was able to classify 54.2% of testing data.


 {'attr5': {0: {'attr6': {0: {'attr2': {0: {'attr1': {0: {'attr4': {0: {'attr3': {0: 0}}, 1: {'attr3': {0: 0}}}}, 1: {'attr4': {0: {'attr3': {0: 0}}, 1: 0}}}}, 1: {'attr4': {0: {'attr3': {0: 0, 1: {'attr1': {0: 0, 1: 0}}}}, 1: {'attr1': {0: {'attr3': {0: 0}}, 1: {'attr3': {0: 0}}}}}}}}, 1: {'attr4': {0: {'attr2': {0: {'attr3': {0: {'attr1': {0: 0}}, 1: {'attr1': {0: 0}}}}, 1: {'attr1': {0: {'attr3': {0: 0}}, 1: {'attr3': {0: 0}}}}}}, 1: {'attr2': {0: {'attr1': {0: {'attr3': {0: 0}}, 1: {'attr3': {0: 0}}}}, 1: {'attr1': {0: {'attr3': {0: 1}}, 1: {'attr3': {0: 0}}}}}}}}}}, 1: {'attr6': {0: {'attr3': {0: {'attr4': {0: {'attr1': {0: {'attr2': {0: 0}}, 1: {'attr2': {0: 0}}}}, 1: {'attr1': {0: {'attr2': {0: 1}}, 1: {'attr2': {0: 0}}}}}}, 1: {'attr2': {0: {'attr4': {0: